In [22]:
#import necessary packages
import pandas as pd
import numpy as np


In [246]:
#read csv
data=pd.read_csv("train.csv")
target='left'
# data1=data.values
# print(data)
keyValueMap={ j:i for i,j in enumerate(list(data.columns.values))}
features=[feature for feature in list(data.columns.values) if feature!=target]
# cFeatures=['Work_accident','promotion_last_5years','sales','salary','satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company']
cFeatures=['Work_accident','promotion_last_5years','sales','salary']
nFeatures=['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company']
# print(len(data.where(data[nFeatures[0]]<=0.46).dropna()))
# nFeatures=['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company']
# print([(feature,np.unique(data[feature])) for feature in features])

In [24]:
def intrinsicCal(q):
    if q:
        return -q*np.log2(q)
    return 0

def entropy(q):
    if q>=1 or q<=0:
        return 0
    return -(q*np.log2(q)+(1-q)*np.log2(1-q))
def gini(q):
    return 2*q*(1-q)
def misClass(q):
    return min(q,1-q)
entropy(0.5)

1.0

In [247]:
#return the information gain of current Attribute with impurity type(entrop/gini/misclass) relative to target attribute
def infoGain(data,currentAttr,impType,targetAttr="left"):
    totalRows=data.shape[0]    
    targetValues, targetValuesCount=np.unique(data[targetAttr],return_counts=True)
    currentValues, currentValuesCount=np.unique(data[currentAttr],return_counts=True)
    targetEntropy=impType(targetValuesCount[0]/(totalRows))
    #(Si/S)*(entropy(target(=0)/Si)),(Si/S)*(entropy(target(=1)/Si)),.......
    weightedEntropy=0
    intrinsic=0
    #(Si/S)*(entropy(target(=0)/Si)),(Sj/S)*(entropy(target(=0)/Sj)),.......
    data1=data.values
    valueMap={ j:0 for j in currentValues}
    i=0
    while(i<totalRows):
        if data1[i][keyValueMap[targetAttr]]==0:
            valueMap[data1[i][keyValueMap[currentAttr]]]+=1
        i+=1
    for j,i in enumerate(currentValues):
        weightedEntropy+= ((currentValuesCount[j]/totalRows)*impType(valueMap[i]/currentValuesCount[j]))
#     for j,i in enumerate(currentValues):
#         weightedEntropy+= ((currentValuesCount[j]/totalRows)*impType(np.unique(data.where(data[currentAttr]==i).dropna()[targetAttr],return_counts=True)[1][0]/currentValuesCount[j]))
#         intrinsic+=intrinsicCal(currentValuesCount[j]/totalRows)
    informationGain=targetEntropy-weightedEntropy
    return informationGain#/intrinsic
print(infoGain(data,'salary',entropy))
# print(infoGain(data,'promotion_last_5years',entropy))
# print(infoGain(data,'Work_accident',entropy))
# print(np.max([infoGain(data,feature,entropy) for feature in cFeatures]))

0.020887530533461263


In [7]:
def numericSplitInfoGain1(data,currentAttr, impType,targetAttr="left" ):
#     print(numericValue)
    totalRows=data.shape[0]
    currentAttrCol=data[currentAttr]
#     print(totalRows)
    targetValues, targetValuesCount=np.unique(data[targetAttr],return_counts=True)
    targetEntropy=impType(targetValuesCount[0]/(totalRows))
    numericValues=[] #all numberic values
    informationGains=[] #for all unique numeric values in data
    for numericValue in np.unique(data[currentAttr]):
#         print(numericValue)
        numericValues.append(numericValue)
        values1,lableCountsForLess=np.unique(data.where(currentAttrCol<=numericValue).dropna()[targetAttr],return_counts=True)
        values2,labelCountsForGreater=np.unique(data.where(currentAttrCol>numericValue).dropna()[targetAttr],return_counts=True)
        recordsLessThanNumeric=np.sum(lableCountsForLess)
        recordsGreaterThanNumeric=np.sum(labelCountsForGreater)
#         print(numericValue,lableCountsForLess,labelCountsForGreater)
        a=0
        b=0
        intrinsic=0
        if recordsLessThanNumeric:
            a=(recordsLessThanNumeric/totalRows)*impType(lableCountsForLess[0]/recordsLessThanNumeric)
            intrinsic+=intrinsicCal(recordsLessThanNumeric/totalRows)
        if recordsGreaterThanNumeric:
            b=(recordsGreaterThanNumeric/totalRows)*impType(labelCountsForGreater[0]/recordsGreaterThanNumeric)
            intrinsic+=intrinsicCal(recordsGreaterThanNumeric/totalRows)
        weightedEntropy=a+b
        informationGain=targetEntropy-weightedEntropy
        informationGains.append(informationGain)
#     print(targetEntropy)
#     print(informationGain)
    return numericValues[np.argmax(informationGains)]
    
# data=data.sort_values(by="satisfaction_level", ascending=True, inplace=False)    
# s=np.unique(data['satisfaction_level'])
# s[np.argmax([numericSplitInfoGain(data,'satisfaction_level',value,entropy) for value in s ])]
numericSplitInfoGain1(data,'satisfaction_level',entropy)
# numericSplitInfoGain(data,'satisfaction_level',0.89,entropy)

0.46

In [51]:
def numericSplitInfoGain(data,currentAttr, impType,target="left"):
    totalRows=data.shape[0]
    data=data.sort_values(by=currentAttr, ascending=True, inplace=False)    
    data1=data.values;
    ones=np.sum(data[target])
    zeros=totalRows-ones
    targetEntropy=impType(ones/(totalRows))
    lableCountsForLess=[0,0]
    labelCountsForGreater=[0,0]
    key=0
    mx=-1
    i=0
    currentIndex=keyValueMap[currentAttr]
    targetIndex=keyValueMap[target]
    while i<totalRows:
        numericValue=data1[i][currentIndex]
        tl=data1[i][targetIndex]
        if tl==0:
            #count of zeros for currentAttr<=numericaval
            lableCountsForLess[0]+=1
        else:
            lableCountsForLess[1]+=1
        
        #count of zeros for currentAttr>numericaval
        labelCountsForGreater[0]=zeros-lableCountsForLess[0]
        labelCountsForGreater[1]=ones-lableCountsForLess[1]
        
        if(i+1!=totalRows and data1[i+1][currentIndex]==numericValue):
            i+=1
            continue

        recordsLessThanNumeric=np.sum(lableCountsForLess)
        recordsGreaterThanNumeric=np.sum(labelCountsForGreater)
        a=0
        b=0
        intrinsic=0
        if recordsLessThanNumeric:
            a=(recordsLessThanNumeric/totalRows)*impType(lableCountsForLess[0]/recordsLessThanNumeric)
            intrinsic+=intrinsicCal(recordsLessThanNumeric/totalRows)
        if recordsGreaterThanNumeric:
            b=(recordsGreaterThanNumeric/totalRows)*impType(labelCountsForGreater[0]/recordsGreaterThanNumeric)
            intrinsic+=intrinsicCal(recordsGreaterThanNumeric/totalRows)
        weightedEntropy=a+b
        informationGain=(targetEntropy-weightedEntropy)
        if(informationGain>=mx):
            mx=informationGain
            key=numericValue
        i+=1
    return (key,mx)
numericSplitInfoGain(data,nFeatures[0],entropy)

(0.46, 0.19312998759326316)

In [248]:
def bestInfoGainFeature(data,cFeatures,impType,targetAttr="left"):
    #info gains for categorical attributes    
    infGains=[infoGain(data,feature,impType) for feature in cFeatures]
    featureIndex=np.argmax(infGains)
    bestFeature=cFeatures[featureIndex]
    return (bestFeature,infGains[featureIndex])
bestInfoGainFeature(data,cFeatures,entropy)

('salary', 0.020887530533461263)

In [238]:
def bestInfoGainFeNumeric(data,impType):
    mxinFo=0
    split=0
    resF=""
    for feature in nFeatures:
        fInfoG=numericSplitInfoGain(data,feature,impType)
#         print("finfo ",feature,fInfoG)
        if fInfoG[1]>=mxinFo:
            resF=feature
            split=fInfoG[0]
            mxinFo=fInfoG[1]
    return (resF,split,mxinFo)
bestInfoGainFeNumeric(data,entropy)

('satisfaction_level', 0.46, 0.19312998759326316)

In [242]:
def buildTree1(filterdData,cFeatures,targetAttr="left"):
    #check if data is pure i.e all tuples have exact 1 type of label
    if(len(np.unique(filterdData[targetAttr]))<=1):
        return np.unique(filterdData[targetAttr])[0]
    #check if all the categorical attributes had been used
    elif(len(cFeatures)==0):
        values,count=np.unique(filterdData[targetAttr],return_counts=True)
        return values[np.argmax(count)]
    #else grow the tree for all possible value of best feature
    #threby filtering the data on that value of best feature
    else:
        #find best feature
        #remove best feature from local list of features(cFeatures) for now
        #create a node with best feature and return it at the end
        #execute a for loop for all possible value of best feature (filtering on this value), and create a branch on
            #this value [best][value1] and receive the result of recursive call on this filtered data      
        bestF=bestInfoGainFeature(filterdData,cFeatures, entropy)[0]
#         print(bestF)
        node={bestF:{}}
        cFeatures=[feature for feature in cFeatures if feature!=bestF]
        for value in np.unique(filterdData[bestF]):
            #filter new datatable based on best feature [value]
            filteredData=filterdData.where(filterdData[bestF]==value).dropna()
            node[bestF][value]=buildTree1(filteredData,cFeatures)
        
    return node
    
# tree=buildTree1(data,cFeatures)
# print(tree)

In [63]:
def buildTree(filteredData,odata,cFeatures,targetAttr="left"):
    if len(filteredData)==0:
        values,count=np.unique(odata[targetAttr],return_counts=True)
        return values[np.argmax(count)]
    #check if data is pure i.e all tuples have exact 1 type of label
    if(len(np.unique(filteredData[targetAttr]))==1):
        return np.unique(filteredData[targetAttr])[0]
    #check if all the categorical attributes had been used
    elif(len(cFeatures)==0):
            bestFNum=bestInfoGainFeNumeric(filteredData, entropy)
            bestF=bestFNum[0]
            split=bestFNum[1]
            ig=bestFNum[2]
            if ig<=0.00000001:
                values,count=np.unique(filteredData[targetAttr],return_counts=True)
                return values[np.argmax(count)]
            node={bestF:{}}
            node[bestF][(split,True)]=buildTree(filteredData.where(filteredData[bestF]<=split).dropna(),odata,cFeatures)
            node[bestF][(split,False)]=buildTree(filteredData.where(filteredData[bestF]>split).dropna(),odata,cFeatures)
            return node;
    else:
        bestFCate=bestInfoGainFeature(filteredData,cFeatures, entropy)
        bestFNum=bestInfoGainFeNumeric(filteredData, entropy)
        isCat=False
        if bestFCate[1]>=bestFNum[2]:
            isCat=True
        if isCat:
            bestF=bestFCate[0]
            node={bestF:{}}
            cFeatures=[feature for feature in cFeatures if feature!=bestF]
            for value in np.unique(filteredData[bestF]):
                #filter new datatable based on best feature [value]
                filteredData=filteredData.where(filteredData[bestF]==value).dropna()
                node[bestF][value]=buildTree(filteredData,odata,cFeatures)
        else:
            bestF=bestFNum[0]
            split=bestFNum[1]
            node={bestF:{}}
            node[bestF][(split,True)]=buildTree(filteredData.where(filteredData[bestF]<=split).dropna(),odata,cFeatures)
            node[bestF][(split,False)]=buildTree(filteredData.where(filteredData[bestF]>split).dropna(),odata,cFeatures)
        
    return node
    
tree=buildTree(data,data,cFeatures)
print(tree)

{'satisfaction_level': {(0.46, True): {'satisfaction_level': {(0.11, True): 1.0, (0.11, False): {'number_project': {(2.0, True): {'last_evaluation': {(0.57, True): {'last_evaluation': {(0.44, True): 0.0, (0.44, False): {'average_montly_hours': {(161.0, True): {'average_montly_hours': {(125.0, True): 0.0, (125.0, False): {'satisfaction_level': {(0.31, True): 0.0, (0.31, False): {'salary': {'high': {'sales': {'IT': 0.0, 'RandD': 0, 'accounting': 0, 'hr': 0, 'marketing': 0, 'product_mng': 0, 'sales': 0, 'support': 0, 'technical': 0}}, 'low': 0, 'medium': 0}}}}}}, (161.0, False): {'average_montly_hours': {(239.0, True): 0.0, (239.0, False): {'sales': {'IT': 0.0, 'sales': 0, 'support': 0, 'technical': 0}}}}}}}}, (0.57, False): {'sales': {'IT': 0.0, 'RandD': 0, 'accounting': 0, 'hr': 0, 'management': 0, 'marketing': 0, 'product_mng': 0, 'sales': 0, 'support': 0, 'technical': 0}}}}, (2.0, False): {'number_project': {(6.0, True): {'average_montly_hours': {(287.0, True): {'sales': {'IT': {'sati

In [250]:
totalDataSets=len(data)
trainDataPercent=100
trainDataSetCount=int(trainDataPercent*totalDataSets/100)

trainData=data[:trainDataSetCount]
validateData=trainData
# validateData=data[trainDataSetCount:]
tree=buildTree1(trainData,cFeatures)
print(tree)
# print("finish")

{'salary': {'high': {'sales': {'IT': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0}}, 1.0: 0.0}}, 'RandD': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'accounting': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'hr': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'management': {'promotion_last_5years': {0.0: {'Work_accident': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'marketing': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'product_mng': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0}}, 1.0: 0.0}}, 'sales': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: 0.0}}, 'support': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0}}, 1.0: 0.0}}, 'technical': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0}}, 1.0: 0.0}}}}, 'low': {'Work_accident': {0.0: {'sales': {'IT':

In [252]:
def predictClass(tree,validateDataRow):
    dataCols=validateDataRow.keys()
    for key in dataCols:
        if key in tree.keys():
            try:
                temp=tree[key][validateDataRow[key]]
            except:
                return 0
            if isinstance(temp,dict):
                return predictClass(temp,validateDataRow)
            else:
                return temp   
count=0
validateData1=validateData.to_dict(orient = "records")
for i in range(len(validateData1)):
    if validateData.iloc[i][keyValueMap["left"]]==predictClass(tree,validateData1[i]):
        count+=1
print(count*100/len(validateData))


76.24132407901762
